### Initialization

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("A:\Projects\Web Scraping Projects\Flipkart Scraping\laptop_data.csv")
df.head()

,Name,Price,Rating,Processor,RAM,OS,Display,Storage,Warranty
0,HP Intel Core i5 11th Gen - (8 GB/512 GB SSD/W...,"₹49,990",4.3,Intel Core i5 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,39.62 cm (15.6 Inch) Display,512 GB SSD,1 Year Onsite Warranty
1,HP (2023) Intel 12th Gen N305 Core i3 - (8 GB/...,"₹41,990",NaN,Intel Core i3 Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,35.56 cm (14 Inch) Display,512 GB SSD,1 Year Onsite Warranty
2,HP 15s (2023) Athlon Dual Core 3050U - (8 GB/5...,"₹31,990",4.0,AMD Athlon Dual Core Processor,8 GB DDR4 RAM,Windows 11 Operating System,39.62 cm (15.6 Inch) Display,512 GB SSD,1 Year Onsite Warranty
3,APPLE 2020 Macbook Air M1 - (8 GB/256 GB SSD/M...,"₹75,990",4.7,Apple M1 Processor,8 GB DDR4 RAM,Mac OS Operating System,33.78 cm (13.3 inch) Display,256 GB SSD,NaN
4,ASUS Vivobook 15 Core i5 11th Gen - (8 GB/512 ...,"₹40,990",4.3,Intel Core i5 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,39.62 cm (15.6 Inch) Display,512 GB SSD,1 Year Onsite Warranty


### Handling Duplicates

In [2]:
df.duplicated().value_counts()

False    558
True     426
dtype: int64

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df = df.reset_index(drop=True) # Reseting the indcies of the rows after deleting duplicate values

In [5]:
df.isnull().sum()

Name           0
Price          0
Rating       193
Processor      0
RAM            0
OS             0
Display        0
Storage      121
Warranty      52
dtype: int64

In [6]:
df.columns = df.columns.str.lower() # Setting the name of columns to lower cases

### Clean Name

In [7]:
df['clean_name'] = df['name'].str.split().str.get(0)

### Clean Prices

In [8]:
df['clean_price'] = df['price'].str.replace('₹', '').str.replace(',', '').astype(int)
df.clean_price.value_counts()

52990     13
62990     12
57990     10
67990     10
109990     8
          ..
46490      1
70430      1
590990     1
85998      1
37390      1
Name: clean_price, Length: 287, dtype: int64

### Clean Rating

In [9]:
df['clean_rating'] = round(df.rating.fillna(df.rating.mean()), 1)

### Clean Processor

In [10]:
df['clean_processor'] = df.processor.str.replace('Processor: ', '').str.replace('Processor-', '')
df['clean_processor2'] = df.name.str.replace('rd', '').str.replace('th', '')

##### 1. Processor Brand

In [11]:
for i, k in zip(df.clean_processor, range(df.shape[0])):
    a = i.split()
    if a[0][0] == 'i':
        df.at[k, 'processor_brand'] = 'Intel'
    elif 'Ryzen' in a:
        df.at[k, 'processor_brand'] = 'AMD'
    else:
        df.at[k, 'processor_brand'] = a[0]

In [12]:
df.processor_brand.value_counts()

Intel       359
AMD         176
Apple        20
MediaTek      3
Name: processor_brand, dtype: int64

##### 2. Processor Core

In [13]:
df['core'] = df.processor.str.replace('Processor', '').str.replace('-', ' ')

In [14]:
for i, k in zip(df.core, range(df.shape[0])):
    a = i.split()
    df.at[k, 'processor_core'] = (' '.join(a[1:3])).replace('Intel', 'Core')

In [15]:
df.processor_core.value_counts()

Core i5              162
Core i7               84
Core i3               79
Ryzen 5               71
Ryzen 7               60
Ryzen 3               25
Core i9               21
Ryzen 9               13
M2                    11
Celeron Dual           9
M2 Pro                 4
Celeron Quad           3
M1 Pro                 3
Athlon Dual            3
MediaTek MT8788        2
M1                     2
Dual Core              1
Ryzen R5               1
MediaTek Kompanio      1
R3 5425U               1
1115G4 upto            1
Pentium Quad           1
Name: processor_core, dtype: int64

In [16]:
df.loc[308, 'processor_core'] = 'Ryzen 3'
df.loc[237, 'processor_core'] = 'Core i3'
df.loc[350, 'processor_core'] = 'Ryzen 5'

In [17]:
for i, k in zip(df.processor_core, range(df.shape[0])):
    a = i.split()
    if a[0] == 'Core':
        df.at[k, 'processor_core'] = str('Intel ' + i)
    else:
        if df.at[k, 'processor_brand'] == 'Intel':
            df.at[k, 'processor_core'] = 'Intel Multi Core'
        elif a[0] != 'Ryzen' and df.at[k, 'processor_brand'] == 'AMD':
            df.at[k, 'processor_core'] = 'Ryzen Multi Core'

In [18]:
df.processor_core.value_counts()

Intel Core i5        162
Intel Core i7         84
Intel Core i3         80
Ryzen 5               72
Ryzen 7               60
Ryzen 3               26
Intel Core i9         21
Intel Multi Core      13
Ryzen 9               13
M2                    11
Ryzen Multi Core       4
M2 Pro                 4
M1 Pro                 3
M1                     2
MediaTek MT8788        2
MediaTek Kompanio      1
Name: processor_core, dtype: int64

##### 3. Processor Generation

In [19]:
for i, k in zip(df.clean_processor2, range(df.shape[0])):
    a = i.split()
    if 'Gen' in a:
        df.at[k, 'generation'] = a[a.index('Gen') - 1]
    else:
        df.at[k, 'generation'] = 0

In [20]:
df.generation = df.generation.astype(int)

In [21]:
df.generation.value_counts()

0     206
11    123
12    122
13     74
10     25
7       3
8       3
6       1
3       1
Name: generation, dtype: int64

### Clean RAM

In [22]:
df['clean_ram'] = df['ram'].apply(lambda x: re.search(r'(\d+)\s*GB', x).group(1)).astype(int)
df.clean_ram.value_counts()

8     265
16    248
32     24
4      18
64      3
Name: clean_ram, dtype: int64

### Clean OS

In [23]:
import re
df['clean_os'] = df['os'].str.replace('Operating System', '').str.replace(r'^OS', '').str.replace('64 bit ', '').str.replace('32 bit ', '')
df.clean_os.value_counts()

Windows 11     475
Windows 10      51
Mac OS          20
Chrome           5
DOS              5
Prime OS         2
Name: clean_os, dtype: int64

### Clean Display

In [24]:
df.display = df.display.str.replace('(', '').str.replace(')', '').str.replace('"', '')

In [25]:
for i, k in zip(df.display, range(df.shape[0])):
    a = i.split()
    if 'Inch' in a:
        df.at[k, 'clean_display'] = str(a[a.index('Inch') - 1])
    elif 'inch' in a:
        df.at[k, 'clean_display'] = str(a[a.index('inch') - 1])
    elif ['inch', 'Inch'] not in a:
        for j in a:
            if j.replace('.', '', 1).isdigit() and 30 < float(j) < 50:
                df.at[k, 'clean_display'] = round(float(j) / 2.54, 1)

In [26]:
df.clean_display = df.clean_display.astype('float')

In [27]:
df.clean_display = round(df.clean_display, 1)

In [28]:
df.clean_display.unique()

array([15.6, 14. , 13.3, 11.6, 16.1, 14.1, 15.4, 14.4, 17.3, 16. , 13.6,
       15. , 18. , 13.5, 13.8, 15.3, 13.4, 14.5, 16.2, 14.2, 17. ])

##### 1. TouchScreen Dispaly

In [29]:
df['touchscreen'] = df.display.apply(lambda x: 1 if 'Touchscreen' in x else 0)

### Clean Storage

In [30]:
df['clean_storage'] = df.name.str.replace('/', ' , ')

##### 1. Clean SSD Storage

In [31]:
for i, k in zip(df.clean_storage, range(df.shape[0])):
    a = i.split()
    if 'SSD' in a:
        if a[a.index('SSD') - 2] < '10':
            df.at[k, 'clean_ssd'] = str(int(a[a.index('SSD') - 2]) * 1024)
        else:
            df.at[k, 'clean_ssd'] = str(a[a.index('SSD') - 2])
    else:
        df.at[k, 'clean_ssd'] = 0

In [32]:
df.clean_ssd = df.clean_ssd.astype(int)

In [33]:
df.clean_ssd.value_counts()

512     383
1024     99
256      51
0        12
2         9
128       3
4         1
Name: clean_ssd, dtype: int64

##### 2. Clean HDD Storage

In [34]:
for i, k in zip(df.clean_storage, range(df.shape[0])):
    a = i.split()
    if 'HDD' in a:
        if a[a.index('HDD') - 2] < '10':
            df.at[k, 'clean_hdd'] = str(int(a[a.index('HDD') - 2]) * 1024)
        else:
            df.at[k, 'clean_hdd'] = str(a[a.index('HDD') - 2])
    else:
        df.at[k, 'clean_hdd'] = str(0)

In [35]:
df.clean_hdd = df.clean_hdd.astype(int)

In [36]:
df.clean_hdd.value_counts()

0       538
1024     20
Name: clean_hdd, dtype: int64

### Clean Warranty

In [37]:
df['clean_warranty'] = df['warranty'].str.split().str.get(0)
df['clean_warranty'].value_counts()

1         421
2          70
3           6
1-year      5
1Year       3
One         1
Name: clean_warranty, dtype: int64

In [38]:
df['clean_warranty'] = df.clean_warranty.replace(['1Year', '1-year', 'One'], ['1', '1', '1'])
df.clean_warranty.value_counts()

1    430
2     70
3      6
Name: clean_warranty, dtype: int64

In [39]:
df.clean_warranty = df.clean_warranty.fillna(df.clean_warranty.mode()[0])
df.clean_warranty.value_counts()

1    482
2     70
3      6
Name: clean_warranty, dtype: int64

In [40]:
df.clean_warranty = df.clean_warranty.astype(int)

In [41]:
df.iloc[:, 9:].head(3)

,clean_name,clean_price,clean_rating,clean_processor,clean_processor2,processor_brand,core,processor_core,generation,clean_ram,clean_os,clean_display,touchscreen,clean_storage,clean_ssd,clean_hdd,clean_warranty
0,HP,49990,4.3,Intel Core i5 Processor (11th Gen),HP Intel Core i5 11 Gen - (8 GB/512 GB SSD/Win...,Intel,Intel Core i5 (11th Gen),Intel Core i5,11,8,Windows 11,15.6,0,"HP Intel Core i5 11th Gen - (8 GB , 512 GB SSD...",512,0,1
1,HP,41990,4.2,Intel Core i3 Processor,HP (2023) Intel 12 Gen N305 Core i3 - (8 GB/51...,Intel,Intel Core i3,Intel Core i3,12,8,Windows 11,14.0,0,HP (2023) Intel 12th Gen N305 Core i3 - (8 GB ...,512,0,1
2,HP,31990,4.0,AMD Athlon Dual Core Processor,HP 15s (2023) Alon Dual Core 3050U - (8 GB/512...,AMD,AMD Athlon Dual Core,Ryzen Multi Core,0,8,Windows 11,15.6,0,"HP 15s (2023) Athlon Dual Core 3050U - (8 GB ,...",512,0,1


### Cleaning Columns

In [42]:
df.drop(columns=['clean_processor', 'clean_processor2', 'core', 'clean_storage'], inplace=True)

In [43]:
df.drop(columns=['name', 'price', 'rating', 'processor', 'ram', 'os', 'display', 'storage', 'warranty'], inplace=True)

In [44]:
df.head(10)

,clean_name,clean_price,clean_rating,processor_brand,processor_core,generation,clean_ram,clean_os,clean_display,touchscreen,clean_ssd,clean_hdd,clean_warranty
0,HP,49990,4.3,Intel,Intel Core i5,11,8,Windows 11,15.6,0,512,0,1
1,HP,41990,4.2,Intel,Intel Core i3,12,8,Windows 11,14.0,0,512,0,1
2,HP,31990,4.0,AMD,Ryzen Multi Core,0,8,Windows 11,15.6,0,512,0,1
3,APPLE,75990,4.7,Apple,M1,0,8,Mac OS,13.3,0,256,0,1
4,ASUS,40990,4.3,Intel,Intel Core i5,11,8,Windows 11,15.6,0,512,0,1
5,ASUS,34990,4.3,Intel,Intel Core i3,11,8,Windows 11,15.6,0,512,0,1
6,Lenovo,33990,4.1,AMD,Ryzen 5,0,8,Windows 11,15.6,0,512,0,1
7,HP,19990,3.7,Intel,Intel Multi Core,0,4,Chrome,14.0,1,0,0,1
8,HP,52990,4.3,Intel,Intel Core i5,11,16,Windows 11,15.6,0,512,0,1
9,Lenovo,35990,4.2,Intel,Intel Core i3,11,8,Windows 11,15.6,0,512,0,2


### Pandas Profiling

In [45]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file = 'ouput.html')

In [46]:
df.to_csv('Clean_Laptop_Data.csv', index=False)  # Set index=False to exclude the index column in the CSV